In [5]:
import requests
from bs4 import BeautifulSoup
#import os
#import time
import datetime
import pandas as pd
#import numpy as np
#import string
import json

##
punctuation=['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'"]

playday = "2021-6-20"
duration = 5
location = '南投市'

now_str = datetime.datetime.now().strftime('%Y-%m-%d')
now = datetime.datetime.strptime(now_str, "%Y-%m-%d")
future = datetime.datetime.strptime(playday, "%Y-%m-%d")
days_diff = (future - now).days
days_end = days_diff + duration

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36'}
#url = 'https://www.accuweather.com/zh/tw/east-district/2515456/daily-weather-forecast/2515456'
url = 'https://www.google.com/search'

my_params = {'q': location+' daily accuweather'}
html_data =requests.get(url=url,headers=headers,params = my_params)
#print(html_data.url)
soup=BeautifulSoup(html_data.text,"html.parser")
#print(soup.prettify())
daily =soup.select_one('div.yuRUbf a')
#print(daily['href'])

daily_data0 = requests.get(url=daily['href']+'?page=0',headers=headers)
print(daily_data0)
daily_data1 = requests.get(url=daily['href']+'?page=1',headers=headers)
daily_data2 = requests.get(url=daily['href']+'?page=2',headers=headers)

soup=BeautifulSoup(daily_data0.text,"html.parser")
title_tag = str(soup.title.text)
print(title_tag)

divs_text=[]

divs = soup.find_all('div', 'daily-wrapper')
for div in divs:
    divs_text.append(str([s for s in div.stripped_strings]))
    
soup=BeautifulSoup(daily_data1.text,"html.parser")
divs = soup.find_all('div', 'daily-wrapper')
for div in divs:
    divs_text.append(str([s for s in div.stripped_strings]))
    
soup=BeautifulSoup(daily_data2.text,"html.parser")
divs = soup.find_all('div', 'daily-wrapper')
for div in divs:
    divs_text.append(str([s for s in div.stripped_strings]))

# jsonArr = json.dumps(divs_text,ensure_ascii=False)
# print(jsonArr)
    
    
## data processing (pandas)

df = pd.DataFrame(data = divs_text)

df.columns = ["day"]

df = df['day'].str.split(',', expand=True)
df = df.drop(4, axis = 1)
df = df.drop(6, axis = 1)

df.columns = ['day','date','H_temp','L_temp',"rain"]

## ultraviolet

Ultraviolet=[]

for days in range(days_diff,days_end):
    url =daily['href']+'?day=%s'%(days+1)
    u_data = requests.get(url=url,headers=headers)
    soup=BeautifulSoup(u_data.text,"html.parser")
    divs = soup.select_one('div.panels div p span').text
    Ultraviolet.append(str(divs))

df1 = pd.DataFrame(data = Ultraviolet)
df1.columns= ['UV']
df1.index = range(days_diff,days_end)
df1=df1.UV.str.extract('(\d+)')
df1.columns= ['UV']

#print(df1)

##
punct = '!"#$&\'/()*+,-.:;<=>?@[\\]^_`{}~'   # `|` is not present here
transtab = str.maketrans(dict.fromkeys(punct, ''))
df['day'] = '|'.join(df['day'].tolist()).translate(transtab).split('|')
df['H_temp'] = '|'.join(df['H_temp'].tolist()).translate(transtab).split('|')
df['L_temp'] = '|'.join(df['L_temp'].tolist()).translate(transtab).split('|')
df['rain'] = '|'.join(df['rain'].tolist()).translate(transtab).split('|')

punct = '\':'
transtab = str.maketrans(dict.fromkeys(punct, ''))
df['date'] = '|'.join(df['date'].tolist()).translate(transtab).split('|')

df=df.iloc[days_diff:days_end]
merged_df = pd.concat([df, df1], axis=1)
merged_df.index = range(len(merged_df))

## pd into dict to json
merged_df_dict = merged_df.to_dict('records')
#print(merged_df_dict)
# merged_df_list= []
# merged_df_list.append(merged_df_dict)
#print(merged_df_list)
jsonArr = json.dumps(merged_df_dict,ensure_ascii=False)
print(jsonArr)


## CSV
# out_dir = './desktop'
# out_name = 'weather_%s.csv'% location
# if not os.path.exists(out_dir):
#     os.mkdir(out_dir)
# fullname = os.path.join(out_dir, out_name)    
# merged_df.to_csv(fullname, encoding='utf_8_sig', index=False) 



<Response [200]>
南投市, 南投縣, 臺灣 每日天氣 | AccuWeather
[{"day": "周日", "date": " 6/20", "H_temp": " 31°", "L_temp": " 25°", "rain": " 52%", "UV": "6"}, {"day": "周一", "date": " 6/21", "H_temp": " 31°", "L_temp": " 25°", "rain": " 56%", "UV": "4"}, {"day": "周二", "date": " 6/22", "H_temp": " 29°", "L_temp": " 24°", "rain": " 66%", "UV": "4"}, {"day": "周三", "date": " 6/23", "H_temp": " 28°", "L_temp": " 23°", "rain": " 88%", "UV": "3"}, {"day": "周四", "date": " 6/24", "H_temp": " 28°", "L_temp": " 24°", "rain": " 85%", "UV": "4"}]


In [18]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import requests
from bs4 import BeautifulSoup
#import os
#import time
import datetime
import pandas as pd
import numpy as np
#import string
import json




##
punctuation=['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'"]

playday = "2021-6-22"
duration = 5
location = 'Renai Township, Nantou County'

now_str = datetime.datetime.now().strftime('%Y-%m-%d')
now = datetime.datetime.strptime(now_str, "%Y-%m-%d")
future = datetime.datetime.strptime(playday, "%Y-%m-%d")
days_diff = (future - now).days
days_end = days_diff + duration

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36'}
url = 'https://www.google.com/search'

my_params = {'q': location+' weather.com/weather/tenday'}
html_data =requests.get(url=url,headers=headers,params = my_params)
#print(html_data.url)
soup=BeautifulSoup(html_data.text,"html.parser")
#print(soup.prettify())
daily =soup.select_one('div.yuRUbf a')
print(daily['href'])

daily_data0 = requests.get(url=daily['href'],headers=headers)
#print(daily_data0.url)

soup=BeautifulSoup(daily_data0.text,"html.parser")
#title_tag = str(soup.title.text)
title_tag = soup.select_one('div.removeIfEmpty h1 span span')
print(title_tag.text)

divs_text=[]
UVs_text=[]

UVs = soup.find_all('span',{'data-testid':'UVIndexValue'})
for UV in UVs:
        UVs_text.append(str([s for s in UV.stripped_strings]))
for i in range(15):
    id = str("titleIndex"+str(i))
    divs = soup.find(id=id)
    for div in divs:
        divs_text.append(str([s for s in div.stripped_strings]))

#print(UVs_text)

index0=[]
index1=[]
index2=[]
index3=[]
index4=[]

for i in range(len(divs_text)):
    if (i%5==0):
        index0.append(divs_text[i])
    elif(i%5==1):
        index1.append(divs_text[i])
    elif(i%5==2):
        index2.append(divs_text[i])
    elif(i%5==3):
        index3.append(divs_text[i])
    elif(i%5==4):
        index4.append(divs_text[i])

## data processing (pandas)

df0 = pd.DataFrame(data = index0)
df1 = pd.DataFrame(data = index1)
df2 = pd.DataFrame(data = index2)
df3 = pd.DataFrame(data = index3)
df4 = pd.DataFrame(data = index4)
df5 = pd.DataFrame(data = UVs_text)
df5.columns = ["d"]


for i in range(30):
    if(i==0 and df5["d"][0]=="['0 of 10']"):
        for i in range(1,30):
            if(i%2==0):
                df5 = df5.drop([i])
    if(i==1 and df5["d"][1]=="['0 of 10']"):
        df5.loc[29] ="['Extreme']"
        df5 = df5.drop([i])
        for i in range(2,30):
            if(i%2==1):
                df5 = df5.drop([i])
df5.index = range(len(df5))


merged_df = pd.concat([df0,df1,df2,df3,df4,df5], axis=1)
#print(merged_df)

merged_df.columns = ["day","temp","weather","rain","other","UV"]
final_df = pd.concat([df0,df1,df2,df3,df4,df5], axis=1)
final_df.columns = ["date","H_temp","L_temp","weather","rain","UV"]


temp0_df = pd.concat([df0,df1], axis=1)
temp0_df.columns = ["0","1"]
temp1_df = pd.concat([df0,df1], axis=1)
temp1_df.columns = ["0","1"]


final_df[['H_temp','L_temp']]= merged_df['temp'].str.split(',', expand=True).iloc[:,[0,2]]
final_df['weather']= merged_df['weather'].str.split(',', expand=True).iloc[:,1]
final_df['rain']= merged_df['rain'].str.split(',', expand=True).iloc[:,1]


##
punct = '!"#$&\'/()*+,-.:;<=>?@[\\]^_`{}~'   # `|` is not present here
transtab = str.maketrans(dict.fromkeys(punct, ''))
merged_df['temp'] = '|'.join(merged_df['temp'].tolist()).translate(transtab).split('|')
#print(merged_df)
final_df['date'] = '|'.join(final_df['date'].tolist()).translate(transtab).split('|')
final_df['H_temp'] = '|'.join(final_df['H_temp'].tolist()).translate(transtab).split('|')
final_df['L_temp'] = '|'.join(final_df['L_temp'].tolist()).translate(transtab).split('|')
final_df['weather'] = '|'.join(final_df['weather'].tolist()).translate(transtab).split('|')
final_df['rain'] = '|'.join(final_df['rain'].tolist()).translate(transtab).split('|')
final_df['UV'] = '|'.join(final_df['UV'].tolist()).translate(transtab).split('|')


temp0_df[['0','1']]= final_df['H_temp'].str.split('°', expand=True)
temp1_df[['0','1']]= final_df['L_temp'].str.split('°', expand=True)

if(temp0_df['1'][0]==None):
    temp0_df['0'][0] = temp1_df['0'][0]


for i in temp0_df.index:
    temp0_df['0'][i] = str(round((int(temp0_df['0'][i])-32)*5/9))
    temp1_df['0'][i] = str(round((int(temp1_df['0'][i])-32)*5/9))
    temp0_df['1'][i] = '°'
    temp1_df['1'][i] = '°'


temp0_df= temp0_df['0'].str.cat(temp0_df['1'],sep="")
temp1_df= temp1_df['0'].str.cat(temp1_df['1'],sep="")
temp0_df.columns =["H_temp"]
temp1_df.columns =["L_temp"]

final_df["H_temp"] = temp0_df
final_df["L_temp"] = temp1_df



final_df= final_df.iloc[days_diff:days_end]

print(final_df)

final_df_dict = final_df.to_dict('records')
jsonArr = json.dumps(final_df_dict,ensure_ascii=False)
#print(jsonArr)



## CSV
# out_dir = './desktop'
# out_name = 'weather_%s.csv'% location
# if not os.path.exists(out_dir):
#     os.mkdir(out_dir)
# fullname = os.path.join(out_dir, out_name)    
# merged_df.to_csv(fullname, encoding='utf_8_sig', index=False) 




https://weather.com/weather/tenday/l/Ren+ai+Nantou+County?canonicalCityId=d6f8f98303cc3b592ece4131eecf7bca35158060dce8be46c7976852a5431116
Renai Township, Nantou
     date H_temp L_temp         weather  rain       UV
2  Mon 21    21°    16°   Thunderstorms   90%  5 of 10
3  Tue 22    21°    16°   Thunderstorms   96%  5 of 10
4  Wed 23    20°    15°   Thunderstorms   98%  5 of 10
5  Thu 24    21°    15°            Rain   78%  8 of 10
6  Fri 25    23°    15°   Thunderstorms   73%  Extreme
